In [ ]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from math import ceil

# Example - Sugar cane production

Harvested sugar cane is immediately transported to a sugar house for further processing, because sugar decreases rapidly trough fermentation.

Suppose we have 11 wagons with sugar, all loaded with the same quantity. Moreover we know the hourly loss of sugar in kg per hour and the remaining live span in hour of the lot in every wagon. (see table below)

We as


This example is from [@gueret1999applications, chap. 6.4].

In [ ]:
data = {
    'lot':{
        1:{'loss': 43, 'life_span': 8},
        2:{'loss': 26, 'life_span': 8},
        3:{'loss': 37, 'life_span': 2},
        4:{'loss': 28, 'life_span': 8},
        5:{'loss': 13, 'life_span': 4},
        6:{'loss': 54, 'life_span': 8},
        7:{'loss': 62, 'life_span': 8},
        8:{'loss': 49, 'life_span': 8},
        9:{'loss': 19, 'life_span': 8},
        10:{'loss': 28, 'life_span':8},
        11:{'loss': 30, 'life_span':8},  
    },
    'number_of_production_lines':3,
    'processing time in h': 2

}

In [ ]:
pd.DataFrame(data['lot'])

,1,2,3,4,5,6,7,8,9,10,11
loss,43,26,37,28,13,54,62,49,19,28,30
life_span,8,8,2,8,4,8,8,8,8,8,8


# algebraic model

- number of time slots: $NS:= ceil(NW/NL)$ (see below)

## sets

## variables

## parameter

- NL number of production lines
- NW number of wagons

In [ ]:
def model(data):
    m = pyo.ConcreteModel('sugar cane production')
    
    # sets and param
    m.wagons = pyo.Set(initialize = data['lot'].keys(), 
                       doc = "set of wagon w")
    
    m.NW = pyo.Param(initialize = max(list(data['lot'].keys())),
                     doc = "number of wagons")
    m.NL = pyo.Param(initialize = data['number_of_production_lines'],
                     doc = "number of production lines")
    m.NS = pyo.Param(initialize = ceil(m.NW / m.NL), 
                    doc = "maximal number of time slots")
    m.slots = pyo.RangeSet(1,m.NS)
    m.duration = pyo.Param(initialize = data['processing time in h'],
                          doc = 'processing time per lot in h')
    
    @m.Param(m.wagons,doc = "hourly loss of wagon w")
    def loss(m,w):
        return data['lot'][w]['loss']
    @m.Param(m.wagons,doc = "life span of wagon w")
    def life(m,w):
        return data['lot'][w]['life_span']
    
    # var
    m.process = pyo.Var(m.wagons, m.slots, domain = pyo.Binary,
                        doc = "1 iff wagon w is assigned to slot s")
    
    # constraints
    @m.Constraint(m.wagons, doc = "each wagon is assigned to one slot")
    def c1(m,w):
        return pyo.quicksum(m.process[w,s] for s in m.slots) == 1
    @m.Constraint(m.slots, doc = "each slot can take maximal NL lots")
    def c2(m,s):
        return pyo.quicksum(m.process[w,s] for w in m.wagons) <= m.NL
    @m.Constraint(m.wagons, doc = 'life span bound per lot')
    def c3(m,w):
        return pyo.quicksum(m.slots[s] * m.process[w,s] for s in m.slots) <= m.life[w] / m.duration
    
    m.OBJ = pyo.Objective(expr = pyo.quicksum(s * m.duration * m.loss[w] * m.process[w,s] 
                                             for s in m.slots for w in m.wagons),
                          sense = pyo.minimize,
                         doc = 'minimize loss of sugar')
    
    # choose and apply solver
    solver = pyo.SolverFactory('glpk')
    solver.solve(m)    
    
    return m

In [ ]:
m = model(data)

print('lost sugar' +str(pyo.value(m.OBJ)))

solution = {
    'assignment': {'lot_'+ str(w): 'slot_' + str(s) for w in m.wagons for s in m.slots if pyo.value(m.process[w,s]) != 0.}
}

solution

lost sugar1602.0


{'assignment': {'lot_1': 'slot_2',
  'lot_2': 'slot_4',
  'lot_3': 'slot_1',
  'lot_4': 'slot_3',
  'lot_5': 'slot_2',
  'lot_6': 'slot_1',
  'lot_7': 'slot_1',
  'lot_8': 'slot_2',
  'lot_9': 'slot_4',
  'lot_10': 'slot_3',
  'lot_11': 'slot_3'}}

In [ ]:
pd.DataFrame(solution)

,assignment
lot_1,slot_2
lot_10,slot_3
lot_11,slot_3
lot_2,slot_4
lot_3,slot_1
lot_4,slot_3
lot_5,slot_2
lot_6,slot_1
lot_7,slot_1
lot_8,slot_2


In [ ]:
m.process.pprint()

process : 1 iff wagon w is assigned to slot s
    Size=44, Index=process_index
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
     (1, 1) :     0 :   0.0 :     1 : False : False : Binary
     (1, 2) :     0 :   1.0 :     1 : False : False : Binary
     (1, 3) :     0 :   0.0 :     1 : False : False : Binary
     (1, 4) :     0 :   0.0 :     1 : False : False : Binary
     (2, 1) :     0 :   0.0 :     1 : False : False : Binary
     (2, 2) :     0 :   0.0 :     1 : False : False : Binary
     (2, 3) :     0 :   0.0 :     1 : False : False : Binary
     (2, 4) :     0 :   1.0 :     1 : False : False : Binary
     (3, 1) :     0 :   1.0 :     1 : False : False : Binary
     (3, 2) :     0 :   0.0 :     1 : False : False : Binary
     (3, 3) :     0 :   0.0 :     1 : False : False : Binary
     (3, 4) :     0 :   0.0 :     1 : False : False : Binary
     (4, 1) :     0 :   0.0 :     1 : False : False : Binary
     (4, 2) :     0 :   0.0 :     1 : False : False : Binary
     (